# Artficial Neural Network_Practice

## Mnist data
#### 본 자료는 정상근 교수님이 수업을 위해서 만든 자료로 학습용 입니다.

In [15]:
'''
Machine Learning Block Implementation Practice

Author : Sangkeun Jung (2019)
'''

# most of the case, you just change the component loading part
# all other parts are almost same
#

from mnist.rsc import load_rsc
from mnist.rsc import convert_to_tensor
from mnist.rsc import make_batch_data

from mnist.nn import mlp as network

import torch 
import torch.nn as nn
import numpy as np

fns = {
        'train' : 
        { 
            'image' : './mnist/data/train.image.npy',
            'label' : './mnist/data/train.label.npy'
        },
        'test' : 
        {
            'image' : './mnist/data/test.image.npy',
            'label' : './mnist/data/test.label.npy'
        },
        'model_fn' : './mnist/trained_model/mlp.model'
      }

def prepare_data(fn_dict, batch_size=100):
    """
        Three main components:
            1. load resource
            2. converting resource as tensor data
            3. batching
    """
    rsc           = load_rsc(fn_dict)
    converted_rsc = convert_to_tensor(rsc)
    batch_data    = make_batch_data(converted_rsc, batch_size)

    return batch_data

def train(model, batch_data, to_model_fn):
    model.train() # set information to pytorch that the current mode is 'training'

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()  
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  

    # training loop
    step = 0
    avg_losses = []
    for epoch in range(10):
        for idx, a_batch in enumerate(batch_data):
            batch_image, batch_label = a_batch
            
            batch_image = torch.tensor(torch.from_numpy(batch_image))
            batch_label = torch.tensor(torch.from_numpy(batch_label).type(torch.LongTensor))


            # init for updating the current batch
            optimizer.zero_grad()

            # forward pass
            predicts = model(batch_image)

            loss = criterion(predicts, batch_label)  # batch_label = reference
            
            # backward pass
            loss.backward() # accumulates the gradient (by addition) for each parameter. This is why you should call optimizer.zero_grad()

            # parameter update 
            optimizer.step()

            # monitoring at every 100 steps
            _loss = loss.item()  # loss.item() # gets the a scalar value held in the loss.
            avg_losses.append( _loss ) 
            if step % 100 == 0 :
                print('Epoch={} \t Step={} \t Loss={:.6f}'.format(
                            epoch, 
                            step,
                            np.mean(avg_losses)
                        )
                      )
                avg_losses = []
            step += 1

    # save model 
    import os
    _dir = os.path.dirname(to_model_fn)
    if not os.path.exists(_dir): os.makedirs(_dir)
    torch.save(model, to_model_fn)
    print("Model saved at {}".format(to_model_fn) )


def test(model, batch_data):
    model.eval() # set information to pytorch that the current mode is 'testing'

    all_predicts   = []
    all_references = []

    for idx, a_batch in enumerate(batch_data):
        batch_image, batch_label = a_batch
            
        batch_image = torch.tensor(torch.from_numpy(batch_image))
        batch_label = torch.tensor(torch.from_numpy(batch_label).type(torch.LongTensor))

        # forward pass
        predicts = model(batch_image) # predicts is logits
        logit = predicts.data.cpu().numpy()

        pred_image_idxs = np.argmax(logit, axis=1)

        for p in pred_image_idxs:all_predicts.append(p)
        for r in batch_label:    all_references.append(r)

    # calculate the accuracy
    num_corrects = 0
    for p,r in zip(all_predicts, all_references):
        if p == r : num_corrects += 1

    accuracy = float(num_corrects) / float( len(all_predicts) )
    print("Accuracy of the model on testing data : {:.6f}".format(accuracy))


if __name__ == '__main__':

    def train_mode():
        batch_data    = prepare_data(fns['train'], batch_size=100)
        model         = network()
        train(model, batch_data, fns['model_fn'])

    def test_mode():
        batch_data = prepare_data(fns['test'], batch_size=100)
        model = torch.load(fns['model_fn'])
        test(model, batch_data)

    train_mode()
    test_mode()





Data loading from ./mnist/data/train.image.npy and ./mnist/data/train.label.npy
60000 Shape of the data


C:\Users\82104\anaconda3\envs\Pytorch\lib\site-packages\ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\82104\anaconda3\envs\Pytorch\lib\site-packages\ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch=0 	 Step=0 	 Loss=2.304047
Epoch=0 	 Step=100 	 Loss=0.630981
Epoch=0 	 Step=200 	 Loss=0.278155
Epoch=0 	 Step=300 	 Loss=0.206600
Epoch=0 	 Step=400 	 Loss=0.191769
Epoch=0 	 Step=500 	 Loss=0.143335
Epoch=1 	 Step=600 	 Loss=0.140900
Epoch=1 	 Step=700 	 Loss=0.124953
Epoch=1 	 Step=800 	 Loss=0.102579
Epoch=1 	 Step=900 	 Loss=0.090647
Epoch=1 	 Step=1000 	 Loss=0.094813
Epoch=1 	 Step=1100 	 Loss=0.075522
Epoch=2 	 Step=1200 	 Loss=0.081327
Epoch=2 	 Step=1300 	 Loss=0.071294
Epoch=2 	 Step=1400 	 Loss=0.059689
Epoch=2 	 Step=1500 	 Loss=0.060757
Epoch=2 	 Step=1600 	 Loss=0.061788
Epoch=2 	 Step=1700 	 Loss=0.046378
Epoch=3 	 Step=1800 	 Loss=0.053027
Epoch=3 	 Step=1900 	 Loss=0.048349
Epoch=3 	 Step=2000 	 Loss=0.037965
Epoch=3 	 Step=2100 	 Loss=0.041846
Epoch=3 	 Step=2200 	 Loss=0.044537
Epoch=3 	 Step=2300 	 Loss=0.037410
Epoch=4 	 Step=2400 	 Loss=0.039103
Epoch=4 	 Step=2500 	 Loss=0.036663
Epoch=4 	 Step=2600 	 Loss=0.025595
Epoch=4 	 Step=2700 	 Loss=0.029032
Epoc

C:\Users\82104\anaconda3\envs\Pytorch\lib\site-packages\ipykernel_launcher.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\82104\anaconda3\envs\Pytorch\lib\site-packages\ipykernel_launcher.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Accuracy of the model on testing data : 0.980100
